In [1]:
import sys
sys.path.insert(0, "/home/jack/github/turtle-backtest")
print(sys.path)

['/home/jack/github/turtle-backtest', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages', '/home/jack/github/turtle-backtest']


In [2]:
from backtesting import Backtest, Strategy
from datetime import datetime
import pandas as pd
import numpy as np

import plotly.graph_objs as go
import plotly.io as pio

# from backtesting.test import GOOG
from turtle.service.data_update import DataUpdate
from turtle.common.enums import TimeFrameUnit
import turtle
print(turtle)

/home/jack/github/turtle-backtest/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

<module 'turtle' from '/home/jack/github/turtle-backtest/turtle/__init__.py'>


In [3]:
class SignalStrategy(Strategy):
    def init(self):
        self.i = -1
        pass        

    def next(self):
        current_signal = self.data.buy_signal[-1] 
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1]*0.985)
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_20[-1] :
            self.position.close()

data_updater = DataUpdate(time_frame_unit=TimeFrameUnit.DAY)

ticker = 'AVGO'
start_date: datetime = datetime(year=2024, month=5, day=1)  
end_date: datetime = datetime(year=2024, month=8, day=30)

count = data_updater.darvas_box_strategy.validate_momentum_all_dates(ticker,start_date, end_date,)
# print(f"{ticker} - {count} trades")
if count > 0: 
    df = data_updater.darvas_box_strategy.df.copy()
    # df.info()
    df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
    df = df.set_index(["hdate"])
    # bt = Backtest(GOOG, SignalStrategy, commission=.002, cash = 10000)
    bt = Backtest(df, SignalStrategy, cash = 10000)
    stats=bt.run()
    # print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean())
    # print(data_updater.darvas_box_strategy.df.iloc[130:140])
    
    if not stats._trades.empty:
        # print(stats._trades.empty)
        print(stats._trades["PnL"].mean(), stats._trades["Duration"].mean())


fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.update_layout(
    title=f"{ticker} Stock OHLC Data",
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,  # Hides the range slider below the chart
    width=2000,  # Set the width
    height=1000,   # Set the height
    xaxis=dict(
        tickformat='%d-%m-%Y',  # Format to show only date
        rangeslider=dict(visible=True, thickness=0.1),  # Add a thin range slider for zoom
        type='date' # Ensure correct axis scaling
    ),
    yaxis=dict(fixedrange=False),
)

for i, trade in stats._trades.iterrows():
    entry_time = trade['EntryTime']
    exit_time = trade['ExitTime']
    entry_price = trade['EntryPrice']
    exit_price = trade['ExitPrice']
    pnl = trade['PnL']

    # Color the line green for winning trades, red for losing trades
    line_color = 'green' if pnl > 0 else 'red'

    # Add a line connecting the buy and sell points
    fig.add_trace(go.Scatter(
        x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='lines+markers',
        marker=dict(color=line_color, size=10),
        line=dict(color=line_color, width=2),
        name=f"Trade {i+1} (PnL: {pnl:.2f})"
    ))

# fig.show()
pio.renderers.default = 'browser'
pio.show(fig)


-4023.64 25 days 00:00:00


gio: http://127.0.0.1:41711: Operation not supported


In [ ]:
class SignalStrategy(Strategy):
    def init(self):
        pass        

    def next(self):
        current_signal = self.data.buy_signal[-1] 
        if current_signal and not self.position:
            # self.buy(sl=self.data.Open[-1])
            self.buy()
        elif self.position and self.data.Close[-1] < self.data.ema_10[-1] :
            self.position.close()
            
df_collect = pd.DataFrame()
data_updater = DataUpdate(time_frame_unit=TimeFrameUnit.DAY)

#symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS50')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS100')
symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS200')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('DARVAS500')
# symbol_group = data_updater.symbol_group_repo.get_symbol_group_list('NAS100')
ticker_list = list(map(lambda x: x.symbol, symbol_group))
# print (ticker_list)
# ticker_list = ['MOD', 'DAKT', 'CRDO', 'ADMA', 'PLTR', 'MSTR', 'SMCI', 'CCL', 'DFH', 'INOD', 'VIST', 'AMRX', 'CLS', 'GGAL', 'UWMC', 'VKTX', 'AMSC', 'CAMT', 'COIN', 'CVNA', 'NU', 'AAOI', 'AVPT', 'FIP', 'GATO', 'MRUS', 'NVDA', 'TDW', 'BBAR', 'BMA', 'CABA', 'MMYT', 'NUVL', 'OII', 'ORN', 'REAX', 'SPOT', 'SUPV', 'TK', 'VST', 'ZETA', 'AVGO', 'CLBT', 'FTAI', 'HCC', 'HLX', 'HOOD', 'IOT', 'MAMA', 'ONON', 'OSCR', 'RKT', 'ROIV', 'TGLS', 'UBER', 'AGI', 'ANF', 'BRBR', 'EGO', 'FTI', 'GSL', 'HMY', 'HRTG', 'MTH', 'MUX', 'ONTO', 'PHM', 'RCL', 'RDNT', 'THC', 'VRT', 'XPO', 'AMD', 'APO', 'BV', 'CUK', 'CVLT', 'GCT', 'LFMD', 'LRCX', 'ORCL', 'PWP', 'SCCO', 'SMMT', 'ACGL', 'ALKT', 'AROC', 'ASC', 'AXON', 'BLDR', 'BX', 'DOLE', 'HUT', 'IREN', 'KKR', 'NXE', 'ROAD', 'TOL', 'WPM', 'ACLS']

start_date: datetime = datetime(year=2023, month=1, day=1)  
end_date: datetime = datetime(year=2024, month=8, day=30)

for ticker in ticker_list:
    if ticker not in ['AVGO']:
        count = data_updater.darvas_box_strategy.validate_momentum_all_dates(ticker, start_date, end_date,)
        # print (ticker, start_date, end_date)
        if count > 0: 
            df = data_updater.darvas_box_strategy.df.copy()
            # print (ticker, len(df), df.info)
            df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'})
            df = df.set_index(["hdate"])
            
            bt = Backtest(df, SignalStrategy, cash = 10000)
            stats=bt.run()
            # print(ticker, stats._trades["PnL"].mean(), stats._trades["Duration"].mean(), len(stats._trades))
        
            if not stats._trades.empty:
                # stats._trades["ticker"] = ticker
                df_collect = pd.concat([df_collect, stats._trades], axis=0)    
if len(df_collect) > 0:
    print(df_collect["PnL"].mean(), df_collect["Duration"].mean(), len(df_collect))
else:
    print("Empty collection")

# print(df_collect.info())

fig = go.Figure()

colors = np.where(df_collect['PnL'] >= 0, 'blue', 'red')
#hover_text = [
#    f"Ticker: {row['ticker']}<br>PnL: {row['PnL']:.2f}<br>EntryPrice: {row['EntryPrice']:.2f}<br>ExitPrice: {row['ExitPrice']:.2f} <br>EntryTime: {row['EntryTime']} <br>ExitTime: {row['ExitTime']}"
#    for _, row in df_collect.iterrows()
#]

#fig.add_trace(go.Scatter(
#    x=df_collect['ExitTime'],  # x-axis: ExitTime
#    y=df_collect['PnL'],       # y-axis: PnL
#    mode='markers',       # Display points only
#    marker=dict(size=8, color=colors),  # Customize the marker
#    text=hover_text,  # Add the hover text
#    hoverinfo='text',  # Show only the custom hover text
#    name='PnL per Trade'
#))

# Update layout for better appearance
fig.update_layout(
    title='PnL vs Exit Time',
    xaxis_title='Exit Time',
    yaxis_title='PnL',
    xaxis=dict(tickformat='%Y-%m-%d',  # Format date and time properly
               showgrid=True),
    yaxis=dict(showgrid=True),
    width=1000,  # Set the width of the plot
    height=600   # Set the height of the plot
)

fig.show()
# pio.renderers.default = 'browser'
# pio.show(fig)
